In [1]:
from tools import *

In [10]:
import import_ipynb
from Model_1 import *

### Model 2a: Minimize Cost Constrained on Customer's Demand and Factory Capacity (Trivial Case)

The trivial case of this model is when the capacity is so large that the program simplifies to Model 1.

#### Input to the Model: 

1. Number of factories and customers
2. Cost vector -> Objective vector
3. Demands vector
4. Factory Capacity

In [11]:
no_I, no_J = 20, 30 # Number of factories, number of customers

cij = np.random.rand(no_I, no_J) # Cost vector
wj = np.random.rand(no_J) # Demands vector
ki = np.random.rand(no_I) + 1000 # Factory Capacity

In [17]:
Wij = np.hstack([np.eye(no_J)]*no_I) # Demand Constraint Matrix
Kij = np.repeat(np.eye(no_I), repeats = no_J, axis = 1) # Capacity Constraint Matrix

c = cij.flatten()

## Standard form of our model

# Upper Bound
A_ub = np.vstack([-Wij, Kij])
b_ub = np.hstack([-wj, ki])

prog_ub = linprog(c, A_ub = A_ub, b_ub = b_ub) # Bigger than or equal constraints

# Equality
A_ub = Kij
b_ub = ki

A_eq = -Wij
b_eq = -wj

prog_eq = linprog(c, A_ub = A_ub, b_ub = b_ub, 
                  A_eq = A_eq, b_eq = b_eq) # Equality constraints

In [13]:
%time prog_ub = linprog(c, A_ub = A_ub, b_ub = b_ub) # ≥ Case

CPU times: user 29.1 ms, sys: 11.6 ms, total: 40.7 ms
Wall time: 10.3 ms


In [14]:
%time prog_eq = linprog(c, A_ub = A_ub, b_ub = b_ub, A_eq = A_eq, b_eq = b_eq) # = Case

CPU times: user 47 ms, sys: 66.8 ms, total: 114 ms
Wall time: 26 ms


In [18]:
np.sum(np.abs(prog_ub.x - prog_eq.x)) # Check that the ≥ case and the = case is identical

1.15105559190363e-08

In [19]:
# Check that the trivial case is the same as model 1
aae(prog_ub.x, optimize_logistic_1(no_I, no_J, cij, wj).x)

### Model 2b: Minimize Cost Constrained on Customer's Demand and Factory Capacity (Non-Trivial Case)

Non-trivial case means that the capacity of each factory cannot cover the volume to all the customers. This also means that we have to check that ∑capacity ≥ ∑demand.

#### Input to the Model: 

1. Number of factories and customers
2. Cost vector -> Objective vector
3. Demands vector
4. Factory Capacity

In [34]:
no_I, no_J = 20, 30 # Number of factories, number of customers

cij = np.random.rand(no_I, no_J) # Cost vector
wj = np.random.rand(no_J) # Demands vector
ki = np.random.rand(no_I) # Factory Capacity

while np.sum(wj) >= np.sum(ki):
    wj = np.random.rand(no_J) # Redo Demands vector
    ki = np.random.rand(no_I) # Redo Factory Capacity

In [35]:
Wij = np.hstack([np.eye(no_J)]*no_I) # Demand Constraint Matrix
Kij = np.repeat(np.eye(no_I), repeats = no_J, axis = 1) # Capacity Constraint Matrix

c = cij.flatten()

## Standard form of our model

# Upper Bound
A_ub = np.vstack([-Wij, Kij])
b_ub = np.hstack([-wj, ki])

prog_ub = linprog(c, A_ub = A_ub, b_ub = b_ub) # Bigger than or equal constraints

# Equality
A_ub = Kij
b_ub = ki

A_eq = -Wij
b_eq = -wj

prog_eq = linprog(c, A_ub = A_ub, b_ub = b_ub, 
                  A_eq = A_eq, b_eq = b_eq) # Equality constraints

In [36]:
np.sum(np.abs(prog_ub.x - prog_eq.x)) # Check that the ≥ case and the = case is identical

1.2124357853149463e-09

In [37]:
# Check that this case is non-trivial
np.all(np.isclose(prog_ub.x, optimize_logistic_1(no_I, no_J, cij, wj).x))

False

#### Modular Code

In [ ]:
def optimize_logistic_2(no_I, no_J, cij, wj, ki):
    
    # Check if the capacity constraints is trivial, solve using model 1
    if np.all(ki >= np.sum(wj)):
        print('Capacity Constraints: Trivial')
        prog = optimize_logistic_1(no_I, no_J, cij, wj)
        
    else:
        print('Capacity Constraints: Non-Trivial')
        Wij = np.hstack([np.eye(no_J)]*no_I) # Demand Constraint Matrix
        Kij = np.repeat(np.eye(no_I), repeats = no_J, axis = 1) # Capacity Constraint Matrix

        c = cij.flatten()

        ## Standard form of our model

        # Upper Bound
        A_ub = np.vstack([-Wij, Kij])
        b_ub = np.hstack([-wj, ki])

        prog = linprog(c, A_ub = A_ub, b_ub = b_ub) # Bigger than or equal constraints
    
    return prog